In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import pickle
import csv


In [14]:
data_path = "/mnt/data/SCALABLE/HARRISON/pca/pca_130_new.csv"
tag_path = "/mnt/data/SCALABLE/HARRISON/pca/tag_list_clean.csv"

In [15]:
#Leggo dataset e carico dati training
arr = pd.read_csv(data_path, sep=",", header=None, index_col=None, dtype="float16")
arr = arr.to_numpy()

points_centroids_map = pickle.load(open('points_centroids_map.pickle', 'rb'))
centroids = pickle.load(open('centroids.pickle', 'rb'))
assignments = pickle.load(open('assignments.pickle', 'rb'))

pca_model = pickle.load(open('pca_model.pickle', 'rb'))
scaler = pickle.load(open('scaler.pickle', 'rb'))



hashtags = []
with open(tag_path, "r") as csvFile: 
    reader = csv.reader(csvFile,delimiter = " ")
    for row in reader:
        hashtags.append(row)

In [4]:
import sys
import torch
import pretrainedmodels.utils as utils
import pretrainedmodels

from io import BytesIO
from PIL import Image

import requests

load_img = utils.LoadImage()
torch.set_printoptions(precision=20)

model_name = 'alexnet'
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

tf_img = utils.TransformImage(model)

def get_vector(image_name):
    if image_name.startswith("http"):
        response = requests.get(image_name)
        input_img = Image.open(BytesIO(response.content))

    else:
        input_img = load_img(image_name)
    input_tensor = tf_img(input_img)
    input_tensor = input_tensor.unsqueeze(0)
    input = torch.autograd.Variable(input_tensor, requires_grad=False)
    output_features = model.features(input).view(4096)
    return output_features.data.numpy()

In [5]:
def get_distances(point, points):
    sP = points
    pA = point
    return np.linalg.norm(sP - pA, ord=2, axis=1.)  # 'distances' is a list

def indexlist_to_points(lst,dataset):
    points = np.empty([len(lst),dataset.shape[1]])
    for index, val in enumerate(lst):
        points[index] = dataset[val]
    return points

In [6]:
image_name = "https://r.hswstatic.com/w_907/gif/tesla-cat.jpg"
k = 15

In [25]:
vector = get_vector(image_name).reshape(1,-1)
sv = scaler.transform(vector)
sample = pca_model.transform(sv)

closest_centroid = np.nanargmin(get_distances(sample, centroids))
print("closest centroid:",closest_centroid)
centroid_points = indexlist_to_points(points_centroids_map[closest_centroid], arr)

nearest_k_images = np.argsort(get_distances(sample,centroid_points))[:k]

nearest_samples = []
for s in nearest_k_images:
    nearest_samples.append(points_centroids_map[closest_centroid][np.asscalar(s)])
    
hash_dict = {}
for n in nearest_samples:
    for h in hashtags[n]:
        if h in hash_dict:
            hash_dict[h] = hash_dict[h] + 1
        else:
            hash_dict[h] = 1

sorted_tags = sorted(hash_dict.items(), key=lambda kv: kv[1], reverse=True)            
for tag in sorted_tags:
    print(tag[0] + " " + str(tag[1]) )

closest centroid: 83
cat 4
bored 4
black 3
mylove 2
home 2
sunday 2
love 2
tired 2
blackandwhite 2
yellow 1
eye 1
truelove 1
picoftheday 1
photooftheday 1
vegan 1
sweet 1
cute 1
handsome 1
awesome 1
man 1
amazing 1
healthy 1
russia 1
catsofinstagram 1
today 1
together 1
lazy 1
boy 1
hand 1
boyfriend 1
happiness 1
couple 1
afternoon 1
funny 1
omg 1
loveyou 1
wild 1
fit 1
fitness 1
friend 1
girl 1
moment 1
night 1
curlyhair 1
sleep 1
gainpost 1
selenagomez 1
followtrain 1
arianagrande 1
justinbieber 1
followforfollow 1
selfie 1
selca 1
sad 1
depressed 1
depression 1
live 1
